In [2]:
!pwd

'pwd' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
%cd ..

c:\Users\haowu\Desktop\Traffic Research\Traffic-Prediction\data_processing


In [4]:
import csv
import pandas as pd
import numpy as np
#import geopandas as gpd
#import networkx as nx
import math
import pickle

import datetime
from datetime import datetime as dt
from collections import Counter
from scipy.spatial.distance import cdist
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from matplotlib import pyplot as plt

from utils import *

### 1. Hyperparameters

In [5]:
# Time
start_date = dt(2022, 2, 14)
end_date = dt(2023, 2, 12)
start_time = 330 # 05:30:00 
end_time = 1260 # 21:00:00
busi_date = pd.bdate_range(start=start_date, end=end_date).date 

In [6]:
'''
execute the following code block only if set_hwd_tmc_segments_shp and set_hwd_xd_segments_str_shp are not in place
'''
# Segments
gdf_tmc_TSMO = gpd.read_file("../data/shape/tmc_shape_TSMO/tmc_shape_TSMO_for_sjoin.geojson")  # 1591 TMC segments in TSMO (selected from Carroll, Howard and Baltimore) used for spatial join
gdf_xd_TSMO = gpd.read_file("../data/shape/xd_shape_TSMO/xd_shape_TSMO_for_sjoin.geojson")  # 2501 XD segments in TSMO (selected from Carroll, Howard and Baltimore) used for spatial join

set_tmc_segments_shp_TSMO = set(gdf_tmc_TSMO.tmc)
set_xd_segments_str_shp_TSMO = set(gdf_xd_TSMO.xd.astype(str))

pickle.dump(set_tmc_segments_shp_TSMO, open("./TSMO/set_tmc_segments_shp_TSMO.pkl", "wb"))
pickle.dump(set_xd_segments_str_shp_TSMO, open("./TSMO/set_xd_segments_str_shp_TSMO.pkl", "wb"))

DriverError: ../data/shape/tmc_shape_TSMO/tmc_shape_TSMO_for_sjoin.geojson: No such file or directory

In [7]:
df_xd_to_tmc_TSMO = pickle.load(open("./TSMO/df_xd_to_tmc_TSMO.pkl", "rb"))
set_xd_segments_str_sjoin_TSMO = set(df_xd_to_tmc_TSMO.id_xd_str)  # 2427

### 2. Downsample Raw Speed Data Files

Execute this code block ONLY IF downsampled csv data are not available on file

tmc process and xd process CANNOT be executed simultaneously, otherwise the notebook will encounter kernel crash


In [7]:
downsampled_tmc_1min_all_bh_part2 = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/B&H_tmc_1min_all_part2.csv", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/downsampled_B&H_tmc_1min_all_part2.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

382it [1:15:25, 11.85s/it]


Finished reading chunks!


In [7]:
# Take ~28 min to execute
# downsampled_tmc_1min_all_bh: shape (203263672, 6)
downsampled_tmc_1min_all_bh_part1 = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/B&H_tmc_1min_all_part1.csv", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/downsampled_B&H_tmc_1min_all_part1.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~2 min to execute
# downsampled_tmc_1min_truck_2022_bh: shape (9509162, 7)
downsampled_tmc_1min_truck_2022_bh = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/B&H_tmc_1min_truck_2022.csv", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/downsampled_B&H_tmc_1min_truck_2022.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~31 sec to execute
# downsampled_tmc_1min_truck_2023_bh: shape (1268097, 7)
downsampled_tmc_1min_truck_2023_bh = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/B&H_tmc_1min_truck_2023.csv", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/downsampled_B&H_tmc_1min_truck_2023.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~4 min to execute
# downsampled_tmc_1min_pv_2022_bh: shape (14498618, 7)
downsampled_tmc_1min_pv_2022_bh = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/B&H_tmc_1min_pv_2022.csv", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/downsampled_B&H_tmc_1min_pv_2022.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~1.5 min to execute
# downsampled_tmc_1min_pv_2023_bh: shape (1860391, 7)
downsampled_tmc_1min_pv_2023_bh = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/B&H_tmc_1min_pv_2023.csv", "/Volumes/Hao's WD/TSMO/Speed/B&H_tmc_1min/downsampled_B&H_tmc_1min_pv_2023.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~5 min to execute
# downsampled_tmc_1min_all_Carroll: shape (5313712, 8)
downsampled_tmc_1min_all_Carroll = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/Carroll_tmc_1min_all.csv", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/downsampled_Carroll_tmc_1min_all.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~4 sec to execute
# downsampled_tmc_1min_truck_2022_Carroll: shape (244643, 7)
downsampled_tmc_1min_truck_2022_Carroll = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/Carroll_tmc_1min_truck_2022.csv", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/downsampled_Carroll_tmc_1min_truck_2022.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~1 sec to execute
# downsampled_tmc_1min_truck_2023_Carroll: shape (33574, 7)
downsampled_tmc_1min_truck_2023_Carroll = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/Carroll_tmc_1min_truck_2023.csv", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/downsampled_Carroll_tmc_1min_truck_2023.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~8 sec to execute
# downsampled_tmc_1min_pv_2022_Carroll: shape (340386, 7)
downsampled_tmc_1min_pv_2022_Carroll = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/Carroll_tmc_1min_pv_2022.csv", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/downsampled_Carroll_tmc_1min_pv_2022.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

# Take ~1 sec to execute
# downsampled_tmc_1min_pv_2023_Carroll: shape (45562, 7)
downsampled_tmc_1min_pv_2023_Carroll = downsample_large_dataset("tmc", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/Carroll_tmc_1min_pv_2023.csv", "/Volumes/Hao's WD/TSMO/Speed/Carroll_tmc_1min/downsampled_Carroll_tmc_1min_pv_2023.csv", set_tmc_segments_shp_TSMO, busi_date, start_time, end_time)

202it [27:58,  8.31s/it]


Finished reading chunks!


### 3. Pivot Downsampled Data and Generate Speed/Density Dataframe
The following code blocks are executed on MSI desktop, which has 64GB RAM. 

#### 3.1 All Speed & Density

In [5]:
downsampled_tmc_1min_all_bh_part1 = pd.read_csv("../data/downsampled_tmc_1min_all_bh/downsampled_B&H_tmc_1min_all_part1.csv")
downsampled_tmc_1min_all_bh_part2 = pd.read_csv("../data/downsampled_tmc_1min_all_bh/downsampled_B&H_tmc_1min_all_part2.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [10]:
df_downsampled_spd_tmc_1min_all_bh_part1 = pivot_df("tmc", "speed", 1, df=downsampled_tmc_1min_all_bh_part1, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_spd_tmc_1min_all_bh_part1.pkl") 
df_downsampled_spd_tmc_1min_all_bh_part2 = pivot_df("tmc", "speed", 1, df=downsampled_tmc_1min_all_bh_part2, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_spd_tmc_1min_all_bh_part2.pkl") 

Check dates of incomplete slots:
2022-03-24 927
2022-04-26 928
2022-05-04 923
2022-05-17 929
2022-05-23 929
2022-06-01 929
2022-07-08 929
2022-07-12 926
2022-07-29 928
2022-08-02 929
2022-08-03 918
2022-08-26 927
2022-10-19 929
Check dates of incomplete slots:
2022-03-24 926
2022-04-14 924
2022-04-21 927
2022-04-22 928
2022-04-25 929
2022-04-28 929
2022-04-29 926
2022-05-02 927
2022-05-03 929
2022-05-04 925
2022-05-06 925
2022-05-09 928
2022-05-10 929
2022-05-11 927
2022-05-12 928
2022-05-16 929
2022-05-18 928
2022-05-20 929
2022-05-23 929
2022-05-24 928
2022-05-25 929
2022-05-26 929
2022-05-27 928
2022-05-30 928
2022-05-31 929
2022-06-01 928
2022-06-02 928
2022-06-03 929
2022-06-06 926
2022-06-15 929
2022-06-16 929
2022-06-17 927
2022-06-22 927
2022-06-23 929
2022-06-24 928
2022-06-28 929
2022-07-01 929
2022-07-29 929
2022-08-02 894
2022-08-03 892
2022-08-16 929
2022-08-17 929
2022-10-03 929
2022-10-12 927
2022-10-13 928
2022-10-14 928
2022-10-17 928
2022-10-19 929
2022-11-17 877
2022

In [14]:
df_downsampled_spd_tmc_1min_all_bh_part1 = pickle.load(open("../data/df_downsampled_spd_tmc_1min_all_bh_part1.pkl", "rb"))  # (241800, 841)
df_downsampled_spd_tmc_1min_all_bh_part2 = pickle.load(open("../data/df_downsampled_spd_tmc_1min_all_bh_part2.pkl", "rb"))  # (241800, 1565)

In [17]:
df_downsampled_spd_tmc_1min_all_bh = pd.concat((df_downsampled_spd_tmc_1min_all_bh_part1, df_downsampled_spd_tmc_1min_all_bh_part2), axis=1) # (241800, 2406)

In [25]:
# Remove duplicated columns
df_downsampled_spd_tmc_1min_all_bh = df_downsampled_spd_tmc_1min_all_bh.loc[:,~df_downsampled_spd_tmc_1min_all_bh.columns.duplicated()].copy()  # (241800, 1565)

In [27]:
pickle.dump(df_downsampled_spd_tmc_1min_all_bh, open("../data/df_downsampled_spd_tmc_1min_all_bh.pkl", "wb"))

In [8]:
# # too large to run on either Mac or MSI
# downsampled_tmc_1min_all_bh = pd.concat((downsampled_tmc_1min_all_bh_part1,downsampled_tmc_1min_all_bh_part2)).reset_index(drop=True)
# downsampled_tmc_1min_all_bh.to_csv("/Volumnes/Hao's WD/TSMO/Speed/B&H_tmc_1min/downsampled_tmc_1min_all_bh.csv", index=False)
# df_downsampled_spd_tmc_1min_all_bh = pivot_df("tmc", "speed", 1, df=downsampled_tmc_1min_all_bh, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_spd_tmc_1min_all_bh.pkl") 

: 

: 

In [18]:
df_downsampled_dens_tmc_1min_all_bh = pivot_df("tmc", "density", 1, df=downsampled_tmc_1min_all_bh, busi_date=busi_date, num_slot=930, freq=1, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_dens_tmc_1min_all_bh.pkl") 

KeyError: 'confidence_score'

#### 3.2 Vehicle-specific Speed & Density

In [35]:
# downsampled_tmc_1min_truck_TSMO: shape (11055476, 7)
downsampled_tmc_1min_truck_TSMO = pd.concat((downsampled_tmc_1min_truck_2022_bh, downsampled_tmc_1min_truck_2023_bh, downsampled_tmc_1min_truck_2022_Carroll, downsampled_tmc_1min_truck_2023_Carroll)).reset_index(drop=True)

# downsampled_tmc_1min_pv_TSMO: shape (16744957, 7)
downsampled_tmc_1min_pv_TSMO = pd.concat((downsampled_tmc_1min_pv_2022_bh, downsampled_tmc_1min_pv_2023_bh, downsampled_tmc_1min_pv_2022_Carroll, downsampled_tmc_1min_pv_2023_Carroll)).reset_index(drop=True)

In [37]:
downsampled_tmc_1min_truck_TSMO.to_csv("/Volumes/Hao's WD/TSMO/Speed/downsampled_tmc_1min_truck_TSMO.csv", index=False)
downsampled_tmc_1min_pv_TSMO.to_csv("/Volumes/Hao's WD/TSMO/Speed/downsampled_tmc_1min_pv_TSMO.csv", index=False)

In [28]:
downsampled_tmc_5min_truck_TSMO = pd.read_csv("/Volumes/Hao's WD/TSMO/Speed/downsampled_tmc_5min_truck_TSMO.csv")
downsampled_tmc_5min_pv_TSMO = pd.read_csv("/Volumes/Hao's WD/TSMO/Speed/downsampled_tmc_5min_pv_TSMO.csv")

In [30]:
df_downsampled_spd_tmc_1min_truck_TSMO = pivot_df("tmc", "speed", 5, df=downsampled_tmc_1min_truck_TSMO, busi_date=busi_date, num_slot=186, freq=5, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_spd_tmc_5min_truck_TSMO.pkl") 
df_downsampled_dens_tmc_1min_truck_TSMO = pivot_df("tmc", "density", 5, df=downsampled_tmc_1min_truck_TSMO, busi_date=busi_date, num_slot=186, freq=5, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_dens_tmc_5min_truck_TSMO.pkl") 
df_downsampled_spd_tmc_1min_pv_TSMO = pivot_df("tmc", "speed", 5, df=downsampled_tmc_1min_pv_TSMO, busi_date=busi_date, num_slot=186, freq=5, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_spd_tmc_5min_pv_TSMO.pkl") 
df_downsampled_dens_tmc_1min_pv_TSMO = pivot_df("tmc", "density", 5, df=downsampled_tmc_1min_pv_TSMO, busi_date=busi_date, num_slot=186, freq=5, start_time=start_time, end_time=end_time, output_file_path="../data/df_downsampled_dens_tmc_5min_pv_TSMO.pkl") 

Check dates of incomplete slots:
Check dates of incomplete slots:
Check dates of incomplete slots:
Check dates of incomplete slots:
